# TDEsMonteMetric


This notebook demonstrates how to use TDEsMonteMetric which calculate what fraction of an input TDE lightcurve would be detected. Paired with a spatial slicer, you can obtain the skymap or the observed light curve. 


In [1]:
%matplotlib inline

import os
import numpy as np
import matplotlib.pylab as plt

import healpy as hp
#import pandas as pd

import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db
import lsst.sims.maf.plots as plots

from lsst.utils import getPackageDir

In [2]:
# import metric from mafContrib
from mafContrib import TDEsMonteMetric

The lightcurve template is stored in an ascii file [TDEfaintfast_z0.1.dat](https://github.com/lsst_nonproject/sims_maf_contrib/data/tde/TDEfaintfast_z0.1.dat).

In [3]:
# light curve file
asciifile = os.path.join(getPackageDir('sims_maf_contrib'), 'data', 'tde', 'TDEfaintfast_z0.1.dat')

# query columns 
mjdCol = 'observationStartMJD'
m5Col = 'fiveSigmaDepth'
filterCol = 'filter'
detectSNR={'u': 5, 'g': 5, 'r': 5, 'i': 5, 'z': 5, 'y': 5}

# set event rate
eventRate = .3

# set light curve parameters
epochStart = -22
peakEpoch = 0
nearPeakT = 10
postPeakT = 14 # two weeks

#requirements for TDE_pop_color
nObsTotal_color = {'u': 0, 'g': 0, 'r': 0, 'i': 0, 'z': 0, 'y': 0}
nObsPrePeak_color = 1
nObsNearPeak_color = {'u': 0, 'g': 0, 'r': 0, 'i': 0, 'z': 0, 'y': 0}
nFiltersNearPeak_color = 3
nObsPostPeak_color = {'u': 0, 'g': 0, 'r': 0, 'i': 0, 'z': 0, 'y': 0} 
nFiltersPostPeak_color = 2

#requirements for TDE_pop_color+u
nObsTotal_u = {'u': 0, 'g': 0, 'r': 0, 'i': 0, 'z': 0, 'y': 0}
nObsPrePeak_u = 1 #1 obs pre-peak is probably quite neccesary though
nObsNearPeak_u = {'u': 1, 'g': 0, 'r': 1, 'i': 0, 'z': 0, 'y': 0}
nFiltersNearPeak_u = 0
nObsPostPeak_u = {'u': 1, 'g': 0, 'r': 1, 'i': 0, 'z': 0, 'y': 0}
nFiltersPostPeak_u = 0



Let's plot the light curve template. 

In [4]:
# plot light curve
def plotlc(dataSlice, time_key='time', mag_key='mag', filter_key='filter', err_key = None,
          peakEpoch=None, nearPeakT=None, postPeakT=None):
    # Input: dataSlice, pandas DataFrame or numpy ndarray
    
    colors = {'u': 'indigo', 'g': 'mediumvioletred', 'r': 'red', 'i': 'orange', 'z': 'limegreen', 'y': 'teal'}
    
    plt.figure(figsize=(10, 6))
    
    for f in np.unique(dataSlice[filter_key]):
        fmatch = dataSlice[filter_key]==f
        time = dataSlice[time_key][fmatch]
        mag = dataSlice[mag_key][fmatch]
        
        # draw a vertical line to denote pre/near/post peak
        if peakEpoch!=None:
            plt.axvline(x=peakEpoch, linestyle='-', linewidth=1, color='k', alpha=0.1)
            plt.axvline(x=peakEpoch-nearPeakT/2, linestyle='--', linewidth=1, color='k', alpha=0.1)
            plt.axvline(x=peakEpoch+nearPeakT/2, linestyle='--', linewidth=1, color='k', alpha=0.1)
            plt.axvline(x=peakEpoch+nearPeakT/2+postPeakT, linestyle='dotted', linewidth=1, color='k', alpha=0.1)
        
        if err_key!=None:
            mag_err = dataSlice[err_key][fmatch]
            plt.errorbar(time, mag, yerr=mag_err, color=colors[f], fmt='.', label=f)
        else:
            plt.scatter(time, mag, color=colors[f], marker='.', label=f)

    plt.legend()
    plt.xlabel('days')
    plt.ylabel('mag')
    plt.ylim(plt.ylim()[::-1])

In [5]:
asciifile = '../../data/tde/TDEfaintfast_z0.1.dat'

lcv_template = np.genfromtxt(asciifile, dtype=[('ph', 'f8'), ('mag', 'f8'), ('flt', 'U1')])

#df = pd.DataFrame()
#df['ph'] = lcv_template['ph']
#df['mag'] = lcv_template['mag']
#df['flt'] = lcv_template['flt']

plotlc(lcv_template, time_key='ph', mag_key='mag', filter_key='flt', 
       peakEpoch=peakEpoch, nearPeakT=nearPeakT, postPeakT=postPeakT)

plt.xlabel('epoch [days] \n An example of an input lightcurve template. \n' )
plt.show()

/Users/yoachim/lsst/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


## Opsim database

Connect to opsim database and get the proposalId

In [6]:
# connect to opsim database
opsdb_baseline = db.OpsimDatabase('baseline_v1.4_10yrs.db')
opsdb_baseline.fetchPropInfo()

({0: 'Other',
  1: 'WFD',
  2: 'DD:290',
  3: 'DD:COSMOS',
  4: 'DD:ECDFS',
  5: 'DD:XMM-LSS',
  6: 'DD:ELAISS1'},
 {'WFD': [1], 'DD': [2, 3, 4, 5, 6], 'NES': []})

In [7]:
# output directory
outDir = 'TDE_example'
resultsDb = db.ResultsDb(outDir=outDir)

In [8]:
# query columns 
mjdCol = 'observationStartMJD'
m5Col = 'fiveSigmaDepth'
filterCol = 'filter'
detectSNR={'u': 5, 'g': 5, 'r': 5, 'i': 5, 'z': 5, 'y': 5}

# set event rate
eventRate = .3

# set light curve parameters
epochStart = -22
peakEpoch = 0
nearPeakT = 10
postPeakT = 14 # two weeks

In [9]:
#requirements for TDE_pop_color
nObsTotal_color = {'u': 0, 'g': 0, 'r': 0, 'i': 0, 'z': 0, 'y': 0}
nObsPrePeak_color = 1
nObsNearPeak_color = {'u': 0, 'g': 0, 'r': 0, 'i': 0, 'z': 0, 'y': 0}
nFiltersNearPeak_color = 3
nObsPostPeak_color = {'u': 0, 'g': 0, 'r': 0, 'i': 0, 'z': 0, 'y': 0} 
nFiltersPostPeak_color = 2

#requirements for TDE_pop_color+u
nObsTotal_u = {'u': 0, 'g': 0, 'r': 0, 'i': 0, 'z': 0, 'y': 0}
nObsPrePeak_u = 1 #1 obs pre-peak is probably quite neccesary though
nObsNearPeak_u = {'u': 1, 'g': 0, 'r': 1, 'i': 0, 'z': 0, 'y': 0}
nFiltersNearPeak_u = 0
nObsPostPeak_u = {'u': 1, 'g': 0, 'r': 1, 'i': 0, 'z': 0, 'y': 0}
nFiltersPostPeak_u = 0


In [10]:
transmetric = TDEsMonteMetric(asciifilelist=[asciifile], mjdCol=mjdCol, m5Col=m5Col, filterCol=filterCol, 
                              detectSNR=detectSNR, eventRate=eventRate,
                              epochStart=epochStart, peakEpoch=peakEpoch, 
                              nearPeakT=nearPeakT, postPeakT=postPeakT,
                              nObsTotal= nObsTotal_color, nObsPrePeak=nObsPrePeak_color, 
                              nObsNearPeak=nObsNearPeak_color, nFiltersNearPeak=nFiltersNearPeak_color, 
                              nObsPostPeak=nObsPostPeak_color, nFiltersPostPeak=nFiltersPostPeak_color,
                              dataout=False)

slicer = slicers.HealpixSlicer(nside=16) 
sqlconstraint = 'night<365' #this is for one year only!

transmetricSky = metricBundles.MetricBundle(transmetric,slicer,sqlconstraint)

group = metricBundles.MetricBundleGroup({'transmetricSky':transmetricSky}, opsdb_baseline, outDir=outDir, 
                                        resultsDb=resultsDb)
group.runAll()
group.plotAll(closefigs=False)

Finish initializing metric
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint night<365 for columns ['fiveSigmaDepth', 'fieldDec', 'fieldRA', 'observationStartMJD', 'filter']
Found 225012 visits
Running:  ['transmetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Plotting figures with "night<365" constraint now.
monopole: 0.0421738  dipole: lon: -169.35, lat: -20.8652, amp: 0.0567985
Plotting complete.
